Minor update to algo3 

1. Create the class graph, tracking the first triples that contribute to the class_graph. These triples are the first part of the approximate bschema.
2. If any connections exist between nodes in the bschema not already represented, add them to the bschema. 
3. Go through each node in the schema, getting the subgraph of the original data graph that is within one hop of the node in the schema.
4. Build a query from this one hop graph to see if it is represented by the bschema. If not, add the nodes in the one hop graph to the schema. (this relies on consistent triple ordering in all the steps of processing the data graph)
5. Optionally, do this in subsequent steps, expanding the bschema until everything is represented. 
6. May use similar approach of binding nodes to candidate nodes to verify the bschema is correct, or can just choose other subgraphs to query against the bschema.

Creating the one hop graph and querying against the bschema finds false negatives, where something should have been represented but hasn't been. 

Binding statements and querying back against the data_graph finds false positives, where it appears that something was represented in the b-schema, but it really wasn't.

Maybe I can just test it both ways. Get the one hop query in the data graph, run it against the bschema, if it is successful, get the one hop query around the subject in the b_schema, with the bind statement, run it against the data graph

Does it make more sense to just bind the subject and see if it's successful or to create the one hop graphs and query them against the other one hop graph. 

skipping the bind for now since it shouldnt matter for one hop graphs, but van add it back

In [1]:
from algo3 import * 
from tqdm import tqdm

In [2]:
data_graph = Graph(store = 'Oxigraph')
data_graph.parse("/Users/lazlopaul/Desktop/223p/experiments/graph-pattern-id/from-data-graph/s223-example.ttl", format="turtle")

<Graph identifier=N91b64dc3ebca4be98059e14fca78b1fc (<class 'rdflib.graph.Graph'>)>

In [28]:

from algo3 import * 
from tqdm import tqdm

# %%
data_graph = Graph(store = 'Oxigraph')
data_graph.parse("/Users/lazlopaul/Desktop/223p/experiments/graph-pattern-id/from-data-graph/s223-example.ttl", format="turtle")

# %%
def copy_graph(g1, g2):
    for s, p, o in g1:
        g2.add((s, p, o))

@dataclass
class BSchemaStatement:
    # NOTE: original_triple maybe always the same as var_triple
    classes: Triple 
    vars: Triple
    original: Triple
    covered_triples: Set[Triple]
    def __init__(self, classes, vars, original):
        self.classes = classes
        self.vars = vars
        self.original = original
        self.covered_triples = set()
    
    def __repr__(self):
        """Better representation for debugging"""
        return f"stmt( classes:{self.classes}, vars {self.vars})"
        
# NOTE: changing triples to bschema statements 
class PatternQuery:
    def __init__(self, triples, graph):
        self.triples = triples
        self.graph = graph
        self.prefixes = get_prefixes(graph)
        self.query_dict = self.make_var_names(triples)
        self.where = self.add_where(self.query_dict)
        self.ask = self.get_ask_query()

    def make_var_names(self, triples):
        query_triples = []
        for stmt in triples:
            class_triple = stmt.classes
            var_triple = stmt.vars
            subject = get_local_name(var_triple[0]).replace("-", "_")
            object = get_local_name(var_triple[2]).replace("-", "_")
            if class_triple[1].startswith("<"):
                p_var = class_triple[1].split("<")[1].split(">")[0]
            else:
                p_var = class_triple[1]
            # TODO: change this down the line probably
            query_triples.append((Triple(class_triple[0], class_triple[1], class_triple[2]), Triple(subject, p_var, object)))
        return query_triples
    
    def add_where(self, query_triples):
        where = []
        added_type_assertions = set()
        
        for klass, var in query_triples:
            # Only add type assertions once per variable
            s_type_key = (var.s, klass.s)
            o_type_key = (var.o, klass.o)
            if s_type_key not in added_type_assertions:
                where.append(f"?{clean_var_name(var.s)} a {convert_to_prefixed(klass.s, self.graph)} .")
                added_type_assertions.add(s_type_key)

            if klass.o != RDFS.Resource:
                if o_type_key not in added_type_assertions:
                    where.append(f"?{clean_var_name(var.o)} a {convert_to_prefixed(klass.o, self.graph)} .")
                    added_type_assertions.add(o_type_key)
                
            if URIRef(var.p) == A:
                continue 

            if URIRef(var.p) in [S223.hasAspect, S223.hasQuantityKind, S223.hasEnumerationKind]:
                # print(f"?{clean_var_name(var.s)} {convert_to_prefixed(URIRef(var.p), self.graph)} {convert_to_prefixed(S223[var.o], self.graph)} .") 
                where.append(f"?{clean_var_name(var.s)} {convert_to_prefixed(URIRef(var.p), self.graph)} {convert_to_prefixed(S223[var.o], self.graph)} .") 
                continue

            where.append(f"?{clean_var_name(var.s)} {convert_to_prefixed(URIRef(var.p), self.graph)} ?{clean_var_name(var.o)} .")
        
        return "\n".join(where)

    def get_ask_query(self):
        query = f"""{self.prefixes}\nASK WHERE {{ {self.where}\n}} LIMIT 1"""
        # print(query)
        return query

# %%
# I don't think this is necessary for the new approach, so just returning node 
def get_or_create_var(node: URIRef, var_counter: Dict = {}) -> str:
        """Create or retrieve variable name for a node"""
        # if node not in var_counter:
        #     var_counter[node] = node  # Store the actual node for variable naming
        return node

def get_class(node: URIRef, data_graph) -> URIRef:
        """Get the class of a node from the data graph"""
        for _, _, o in data_graph.triples((node, A, None)):
            return o
        # # if the node is a class just return the class
        # for _, _, o in data_graph.triples((None, RDF.type, node)):
        #     return node
        # Default to resource, since I"m putting these results back in a graph
        return URIRef("http://www.w3.org/2000/01/rdf-schema#Resource")

# NOTE: updated to create bschema patterns
def create_class_pattern(triple: Tuple[URIRef, URIRef, URIRef], data_graph) -> Tuple[URIRef, URIRef, URIRef]:
        """Create class-based pattern from concrete triples"""
        pattern = []
        # NOTE: not used, should delete if this works
        var_counter = {}
        
        # Get classes
        s_class = get_class(triple[0], data_graph)
        o_class = get_class(triple[2], data_graph)
        
        # # Create variable names
        # s_var = get_or_create_var(triple[0], var_counter)
        # o_var = get_or_create_var(triple[2], var_counter)
        
        # class_triple = Triple(str(s_class), str(triple[1]), str(o_class))
        # var_triple = Triple(s_var, triple[1], o_var)
        # original_triple = Triple(triple[0], triple[1], triple[2])

        return (s_class, triple[1], o_class)
        
        # return BSchemaStatement(class_triple, var_triple, original_triple)

# %%
def get_subgraph_with_hops(
    graph: Graph, 
    central_node: Union[str, rdflib.URIRef], 
    num_hops: int,
    get_classes = True
) -> Graph:
    """
    Extract a subgraph containing all triples within num_hops from central_node,
    including rdf:type information for all entities.
    
    Args:
        graph: The source RDF graph
        central_node: The central node URI (as string or URIRef)
        num_hops: Number of hops to traverse from the central node
        get_classes: put the classes for everything in the graph
    
    Returns:
        A new Graph containing the subgraph
    """
    if isinstance(central_node, str):
        # print("central node is string: ", central_node)
        central_node = rdflib.URIRef(central_node)
    
    subgraph = Graph(store = 'Oxigraph')
    visited_nodes = set()
    current_layer = {central_node}
    
    # Add class information for central node
    for class_uri in graph.objects(central_node, RDF.type):
        subgraph.add((central_node, RDF.type, class_uri))
    
    for hop in range(num_hops):
        next_layer = set()
        
        for node in current_layer:
            if node in visited_nodes:
                continue
            visited_nodes.add(node)
            
            # Get triples where node is subject
            for p, o in graph.predicate_objects(node):
                subgraph.add((node, p, o))
                if isinstance(o, rdflib.URIRef):
                    next_layer.add(o)
                    # Add class information for object
                    for class_uri in graph.objects(o, RDF.type):
                        subgraph.add((o, RDF.type, class_uri))
            
            # Get triples where node is object
            for s, p in graph.subject_predicates(node):
                subgraph.add((s, p, node))
                if isinstance(s, rdflib.URIRef):
                    next_layer.add(s)
                    # Add class information for subject
                    for class_uri in graph.objects(s, RDF.type):
                        subgraph.add((s, RDF.type, class_uri))
        
        current_layer = next_layer
    if get_classes:
        for s, p, o in subgraph:
            for class_uri in graph.objects(s, RDF.type):
                subgraph.add((s, RDF.type, class_uri))
            for class_uri in graph.objects(o, RDF.type):
                subgraph.add((o, RDF.type, class_uri))
    
    return subgraph

# %%
def create_class_graph(data_graph: Graph):
    class_graph = Graph(store = 'Oxigraph')
    triple_graph = Graph(store = 'Oxigraph')
    triple_schema = []
    
    for triple in data_graph: 
        class_triple = create_class_pattern(triple, data_graph)
        if class_triple in class_graph:
            continue
        else:
            class_graph.add(class_triple)
            triple_graph.add(triple)
            stmt = BSchemaStatement(class_triple, triple, triple)
            triple_schema.append(stmt)
            # if not ('#' in triple[2]):
            #     print('TRIPLE SHOULD BE NAMESPACED')
    # add any triples to triple_graph connecting things already present 
    all_nodes = set([s for s, p, o in triple_graph] + [o for s, p, o in triple_graph])
    for node in all_nodes:
        for node2 in all_nodes:
            for triple in data_graph.triples((node, None, node2)):
                if triple in triple_graph:
                    continue
                else:
                    triple_graph.add(triple)
                    # print('adding triple: ', triple)
    return class_graph, triple_schema, triple_graph

# %%
def compare_to_query(subject, data_graph, triple_graph, query_hops = 1):
    one_hop_graph = get_subgraph_with_hops(data_graph, subject, query_hops)
    _, one_hop_triple_schema, one_hop_triple_graph = create_class_graph(one_hop_graph)

    # all_nodes = set()
    # for s, p, o in one_hop_triple_graph:
    #     all_nodes.add(s)
    #     all_nodes.add(o)
    # for node in list(all_nodes):
    #     if not ('#' in node):
    #         print(node)
    pq = PatternQuery(one_hop_triple_schema, one_hop_graph)
    
    # print(pq.get_ask_query())
    res = triple_graph.query(pq.get_ask_query())
    # print(res.askAnswer)
    if not res.askAnswer:
        # (one_hop_graph - triple_graph).print()
        return False, pq, one_hop_graph
    return True, pq, one_hop_graph

def bind_triples_to_query(subject, subject_class, pq: PatternQuery):
    """Bind triples' values to query variables. 
    All triples have the same subject and object, so we only bind once."""
    
    candidate_stmts = pq.triples
    var_bindings = []
    
    for stmt in candidate_stmts:
        if stmt.classes[0] == subject_class:
            var_bindings.append(clean_var_name(get_local_name(stmt.vars[0])))
    
    if len(var_bindings) == 0:
        # print(f"[BIND] ERROR: Could not find matching pattern for binding!")
        return False
        
    for s_var in var_bindings:
        query = pq.get_ask_query()
        values_clause = f"VALUES (?{s_var}) {{ (<{subject}>) }}"
        
        # print(f"[BIND] VALUES clause: {values_clause}")
        
        # Insert VALUES clause into WHERE clause
        where_pos = query.find("WHERE")
        if where_pos != -1:
            insert_pos = query.find("{", where_pos) + 1
            query = query[:insert_pos] + "\n    " + values_clause + "\n" + query[insert_pos:]
        
        # print(f"[BIND] Final query:\n{query}")
        yield query


# %%
#DEBUG
failed_subjects = []
failed_queries = []
failed_graphs = []
check_1_fails = []
def compare_all_nodes(data_graph, triple_graph, add_hops = 10):
    matches = 0
    not_matches = 0
    unmatched_subjects = set()
    added_to_triples = 0
    print('Starting Graph Types')
    unmatched_graph = Graph()
    print(f"{triple_graph.store=}, {data_graph.store=}")
    # Add in 'concentric circles' from starting graph
    # adding from 10 concentric circles
    seen_subjects = set()
    seen_3_9 = False
    for s, p, o in tqdm(data_graph):
        if s in seen_subjects:
            continue
        seen_subjects.add(s)
        success, pq, one_hop_graph = compare_to_query(subject = s, data_graph = data_graph, triple_graph = triple_graph)
        if s == URIRef('urn:example#vav-cooling-only_name_3_9'):
            seen_3_9 = True
            print(' for ', s)
            print('success ', success)
            print(pq.get_ask_query())
            print('-'* 20)
            one_hop_graph.print()
            print('-'* 20)
            # triple_graph.print()
        if s == URIRef('urn:example#multiple-zone-ahu_name_3'):
            seen_3_9 = True
            print(' for ', s)
            print('success ', success)
            print(pq.get_ask_query())
            print('-'* 20)
            one_hop_graph.print()
            print('-'* 20)
            # triple_graph.print()
        if ("urn:example#vav-cooling-only_name_" in str(s)) & (not seen_3_9):
            print('SAW DIFFERENT VAV BEFORE 3_9: ', s)
        if success == False:
            check_1_fails.append([s, pq.get_ask_query(), triple_graph])
        if success:
            #ask the other way
            s_class = get_class(s, data_graph)

            one_hop_triple_graph = get_subgraph_with_hops(triple_graph, s, 1)
            _, one_hop_triple_schema, _ = create_class_graph(one_hop_triple_graph)
            pq = PatternQuery(one_hop_triple_schema, one_hop_triple_graph)
            bound_queries = bind_triples_to_query(s, s_class, pq)
            check_2_success = False
            query_debug = ""
            for query in bound_queries:
                # any bound query success means success
                # querying data_graph
                if data_graph.query(query).askAnswer:
                    check_2_success = True
                else:
                    failed_queries.append(query)
                    failed_graphs.append(one_hop_graph)
                # print(pq.get_ask_query())
            if s == URIRef('urn:example#vav-cooling-only_name_3_9'):
                print(' for ', s)
                print('success check 2 ', check_2_success)
                print(pq.get_ask_query())
                print('-'* 20)
                one_hop_graph.print()
                print('-'* 20)
                one_hop_triple_graph.print()
                # triple_graph.print()
            if s == URIRef('urn:example#multiple-zone-ahu_name_3'):
                seen_3_9 = True
                print(' for ', s)
                print('success ', success)
                print(pq.get_ask_query())
                print('-'* 20)
                one_hop_graph.print()
                print('-'* 20)
                # triple_graph.print()
        if (check_2_success & success)==False:
            failed_subjects.append([s, success, check_2_success])
            # more messiness, but don't add classes
            for s,p,o in one_hop_graph:
                # if p == A:
                #     continue
                # added_triples_graph.add((s,p,o))
                # adding to triples graph here
                triple_graph.add((s,p,o))
            added_to_triples += 1
        
    print('Graph Types After query_comparison')
    print(f"{triple_graph.store=}, {data_graph.store=}, {unmatched_graph=}")
    return matches, not_matches, unmatched_subjects, added_to_triples, triple_graph


In [29]:

data_graph = Graph(store = 'Oxigraph')
data_graph.parse("/Users/lazlopaul/Desktop/223p/experiments/graph-pattern-id/from-data-graph/brick-example.ttl", format="turtle")
# need to remove empty subject 
remove_triples = []
for s, p, o in data_graph:
    if str(get_local_name(s)).strip() == "":
        # print("skipping subject empty string")
        remove_triples.append((s,p,o))
for triple in remove_triples:
    data_graph.remove(triple)
        
class_graph, triple_schema, triple_graph = create_class_graph(data_graph)
matches, not_matches, unmatched_subjects, added_to_triples, new_triple_graph = compare_all_nodes(data_graph = data_graph, triple_graph = triple_graph, add_hops = 2)
# TODO: fix this excess class addition issue 
for s, p, o in new_triple_graph:
    if p == A:
        if (None, None, s) in new_triple_graph:
            continue
        else:
            new_triple_graph.remove((s,p,o))
new_triple_graph.serialize('algo4-brick.ttl', format = 'ttl')

Starting Graph Types
triple_graph.store=<oxrdflib.store.OxigraphStore object at 0x10d132da0>, data_graph.store=<oxrdflib.store.OxigraphStore object at 0x10d131150>


 57%|█████▋    | 1949/3428 [00:01<00:00, 1638.03it/s]

 for  urn:example#vav-cooling-only_name_3_9
success  True
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: 

 95%|█████████▍| 3256/3428 [00:02<00:00, 1503.36it/s]

 for  urn:example#multiple-zone-ahu_name_3
success  True
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <

100%|██████████| 3428/3428 [00:02<00:00, 1589.30it/s]


--------------------
Graph Types After query_comparison
triple_graph.store=<oxrdflib.store.OxigraphStore object at 0x10d132da0>, data_graph.store=<oxrdflib.store.OxigraphStore object at 0x10d131150>, unmatched_graph=<Graph identifier=N0855be9fba49479dafa53e5bac9f4a4c (<class 'rdflib.graph.Graph'>)>


<Graph identifier=Na37e1c4cdd5348ab800a30f89ba47df6 (<class 'rdflib.graph.Graph'>)>

In [5]:
for ahu, p, o in data_graph:
    if o == BRICK.AHU:
        print(ahu)
        break

for vav, p, o in data_graph:
    if o == BRICK.VAV:
        if len(list(data_graph[vav: BRICK.hasPart: None])) < 2:
            print(vav)
            break

urn:example#multiple-zone-ahu_name_3
urn:example#vav-cooling-only_name_3_9


In [6]:
list(data_graph.triples((ahu, None, vav)))

[(rdflib.term.URIRef('urn:example#multiple-zone-ahu_name_3'),
  rdflib.term.URIRef('https://brickschema.org/schema/Brick#feeds'),
  rdflib.term.URIRef('urn:example#vav-cooling-only_name_3_9'))]

In [7]:
class_graph, triple_schema, triple_graph = create_class_graph(data_graph)
triple_graph.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:example#multiple-zone-ahu_name_3> a brick:AHU ;
    rdfs:label "AHU"^^xsd:string ;
    brick:feeds <urn:example#vav-with-reheat_name_3_9> ;
    brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_3>,
        <urn:example#multiple-zone-ahu_htg_coil_3>,
        <urn:example#multiple-zone-ahu_ra_damper_3>,
        <urn:example#multiple-zone-ahu_sa_fan_3> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd_3>,
        <urn:example#multiple-zone-ahu_ma_temp_3>,
        <urn:example#multiple-zone-ahu_oa_temp_3>,
        <urn:example#multiple-zone-ahu_ra_temp_3>,
        <urn:example#multiple-zone-ahu_sa_temp_3> .

<urn:example#multiple-zone-ahu_clg_coil-valve_cmd_3> a brick:Valve_Position_Command ;
    rdfs:label "Valve_Position_Command"^^xsd:string .

<urn:example#multiple-zone-ahu_clg_coil_3> a brick:Cooling_Co

In [8]:
print("Matched: ", matches, " Not matched: ", not_matches, "Unique unmatched subjects: ", len(unmatched_subjects))
print("Added to triples_graph: ", added_to_triples, " times")

Matched:  0  Not matched:  0 Unique unmatched subjects:  0
Added to triples_graph:  801  times


In [9]:
for f in check_1_fails:
    query = f[1]
    i = query.find('WHERE')
    if i != -1:
        print(query[i:])
        # print(query[i:])

WHERE { ?multiple_zone_ahu_name_3 a <https://brickschema.org/schema/Brick#AHU> .
?multiple_zone_ahu_ra_damper_3 a <https://brickschema.org/schema/Brick#Damper> .
?multiple_zone_ahu_name_3 <https://brickschema.org/schema/Brick#hasPart> ?multiple_zone_ahu_ra_damper_3 .
?multiple_zone_ahu_ra_damper_3 <http://www.w3.org/2000/01/rdf-schema#label> ?Damper .
?multiple_zone_ahu_ra_damper_dmppos_3 a <https://brickschema.org/schema/Brick#Damper_Position_Command> .
?multiple_zone_ahu_ra_damper_3 <https://brickschema.org/schema/Brick#hasPoint> ?multiple_zone_ahu_ra_damper_dmppos_3 .
} LIMIT 1


In [10]:
f[2].query(f[1]).askAnswer

False

In [11]:
query = """ASK WHERE { ?multiple_zone_ahu_name_1 a <https://brickschema.org/schema/Brick#AHU> .
?multiple_zone_ahu_ra_damper_1 a <https://brickschema.org/schema/Brick#Damper> .
?multiple_zone_ahu_name_1 <https://brickschema.org/schema/Brick#hasPart> ?multiple_zone_ahu_ra_damper_1 .
?multiple_zone_ahu_ra_damper_1 <http://www.w3.org/2000/01/rdf-schema#label> ?Damper .
?multiple_zone_ahu_ra_damper_dmppos_1 a <https://brickschema.org/schema/Brick#Damper_Position_Command> .
# ?multiple_zone_ahu_ra_damper_1 brick:hasPoint ?multiple_zone_ahu_ra_damper_dmppos_1 .
} LIMIT 1"""
f[2].query(query).askAnswer

False

In [12]:
f[2].print()

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:example#multiple-zone-ahu_name_0> brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_0>,
        <urn:example#multiple-zone-ahu_htg_coil_0>,
        <urn:example#multiple-zone-ahu_ra_damper_0>,
        <urn:example#multiple-zone-ahu_sa_fan_0> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd_0>,
        <urn:example#multiple-zone-ahu_ma_temp_0>,
        <urn:example#multiple-zone-ahu_oa_temp_0>,
        <urn:example#multiple-zone-ahu_ra_temp_0>,
        <urn:example#multiple-zone-ahu_sa_temp_0> .

<urn:example#multiple-zone-ahu_name_1> brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_1>,
        <urn:example#multiple-zone-ahu_htg_coil_1>,
        <urn:example#multiple-zone-ahu_ra_damper_1>,
        <urn:example#multiple-zone-ahu_sa_fan_1> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd

In [13]:
# ra_damper only thing failing check_1

for s in failed_subjects:
    if s[2] == False:
        print(s)

[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_8'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_7'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_6'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_5'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_4'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_3'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_2'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_14'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_13'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_12'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_11'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_10'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-reheat_rhc_3_1'), True, False]
[rdflib.term.URIRef('urn:example#vav-with-rehe

In [14]:
for query, graph in zip(failed_queries, failed_graphs):
    i = query.find('VALUES')
    if i != -1:
        print(query[i:i+100])
        # print(query[i:])
        # print(query)
        # graph.print()
        # break

In [15]:
g = Graph(store = 'Oxigraph')
g.parse(data = """@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:example#vav-with-reheat_name-dmp_3_9> a brick:Damper ;
    brick:hasPoint <urn:example#vav-with-reheat_name-dmp-dmppos_3_9> .

<urn:example#vav-with-reheat_name-dmp-dmppos_3_9> a brick:Damper_Position_Command ;
    rdfs:label "Damper_Position_Command"^^xsd:string .""", format = 'ttl')

g.query(query).askAnswer

False

In [16]:
print("Matched: ", matches, " Not matched: ", not_matches, "Unique unmatched subjects: ", len(unmatched_subjects))
print("Added to triples_graph: ", added_to_triples, " times")

Matched:  0  Not matched:  0 Unique unmatched subjects:  0
Added to triples_graph:  801  times


In [17]:
triple_graph.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:example#multiple-zone-ahu_name_3> a brick:AHU ;
    rdfs:label "AHU"^^xsd:string ;
    brick:feeds <urn:example#vav-with-reheat_name_3_9> ;
    brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_3>,
        <urn:example#multiple-zone-ahu_htg_coil_3>,
        <urn:example#multiple-zone-ahu_ra_damper_3>,
        <urn:example#multiple-zone-ahu_sa_fan_3> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd_3>,
        <urn:example#multiple-zone-ahu_ma_temp_3>,
        <urn:example#multiple-zone-ahu_oa_temp_3>,
        <urn:example#multiple-zone-ahu_ra_temp_3>,
        <urn:example#multiple-zone-ahu_sa_temp_3> .

<urn:example#multiple-zone-ahu_clg_coil-valve_cmd_3> a brick:Valve_Position_Command ;
    rdfs:label "Valve_Position_Command"^^xsd:string .

<urn:example#multiple-zone-ahu_clg_coil_3> a brick:Cooling_Co

In [18]:
for s, p, o in new_triple_graph:
    s_class = get_class(s, new_triple_graph)
    pq = PatternQuery(triple_schema, new_triple_graph)
    # print(pq.get_ask_query())
    bound_queries = bind_triples_to_query(s, s_class, pq)
    break

    
for query in bound_queries:
    # print(query)
    print(new_triple_graph.query(query).askAnswer)

False
False
False


In [19]:

s223_data_graph = Graph(store = 'Oxigraph')
s223_data_graph.parse("/Users/lazlopaul/Desktop/223p/experiments/graph-pattern-id/from-data-graph/s223-example.ttl", format="turtle")
s223_bschema = BSchemaGenerator(s223_data_graph)
s223_class_graph, s223_triple_schema, s223_triple_graph = create_class_graph(s223_data_graph)

skipping subject empty string
